In [15]:
import random

def generateData(machines=4, jobs=7, seed=0):
    PROC = [ [ 0 for m in range(machines) ] for j in range(jobs) ]
    random.seed(seed)
    for j in range(jobs):
        for m in range(machines):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [16]:
import pulp

def IP(PROC):

    steps = [ len(job) for job in PROC ]
    assert(min(steps)==max(steps))
    N_MACHINES = len(PROC[0])
    N_JOBS = len(PROC)


    def val(x):
        return int(pulp.value(x))

    def proc(job, machine):
        return pulp.lpSum( [ PROC[j][machine] * JJ[job][j] for j in range(N_JOBS) ] ) 

    JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)), lowBound=0, upBound=1, cat='Integer')
    Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')

    prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)
    prob += Stop[N_JOBS-1][N_MACHINES-1]

    # JJ is a permutation of the jobs
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
        prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            prob += Start[j][m] + proc(j,m) == Stop[j][m]

    solvers = pulp.listSolvers(onlyAvailable=True) 
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
                
    return acc, int(pulp.value(prob.objective))

In [17]:
import numpy as np

def totalTime(PROC, seq, log=False):
    
    def isPermutation(seq):
        for i in range(len(seq)):
            if i not in seq:
                return False
        return True
    
    steps = [ len(job) for job in PROC ]
    assert(len(PROC) == len(seq))
    assert(isPermutation(seq))
    assert(min(steps)==max(steps))
    n_machines = len(PROC[0])
    n_jobs = len(PROC)
    wait = np.zeros([n_jobs, n_machines], dtype=int)
    idle = np.zeros([n_jobs, n_machines], dtype=int)
    start = np.zeros([n_jobs, n_machines], dtype=int)
    stop  = np.zeros([n_jobs, n_machines], dtype=int)
    proc = np.zeros([n_jobs, n_machines], dtype=int)
    for job in range(n_jobs):
        proc[job] = PROC[seq[job]]
    for job in range(n_jobs):
        for machine in range(n_machines):
            start[job, machine] = max(stop[job-1, machine] if job>0 else 0, stop[job, machine-1] if machine>0 else 0)
            wait[job, machine] = start[job, machine] - (stop[job, machine-1] if machine>0 else 0)
            idle[job, machine] = start[job, machine] - (stop[job-1, machine] if job>0 else 0)
            stop[job, machine] = start[job, machine] + proc[job, machine]

    if log:

        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)
        
        row = '|         |'
        for m in range(n_machines):
            row += f"            |" \
                    f" Machine: {m:1d} |"
        print(row)
        
        for j in range(n_jobs):
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Idle: {idle[j,m]:4d} |"
            print(row)
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Start:{start[j,m]:4d} |"  
            print(row)
            
            row = f'| Job: {seq[j]:2d} |'
            for m in range(n_machines):
                row += f" Wait: {wait[j,m]:4d} | " \
                       f"Proc: {proc[j,m]:4d} |"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Stop: {stop[j,m]:4d} |"  
            print(row)
            
        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)

    
    return stop[n_jobs-1, n_machines-1]                                                       

In [18]:
PROC = generateData(machines=4, jobs=7, seed=9850)

Compute the optimal job schedule:

In [19]:
seq, proctime = IP(PROC)
print(seq)
print(proctime)

[5, 4, 3, 2, 0, 6, 1]
54


Check the processing time and print the schedule

In [20]:
totalTime(PROC, seq)

54

In [21]:
totalTime(PROC, seq, log=True)

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    8 |            | Idle:   12 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   8 |            | Start:  12 |
| Job:  5 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    7 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    5 |
|         |            | Stop:    1 |            | Stop:    8 |            | Stop:   12 |            | Stop:   17 |
|---------|------------|------------|------------|------------|---------

54

# Implementing Greedy Solution

In [22]:
def greedy_algorithm(PROC):
    # Number of jobs and machines
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])

    # Greedy algorithm: sort jobs by total processing time (shortest first)
    job_order = sorted(range(N_JOBS), key=lambda j: sum(PROC[j]))
    
    # Return the order of jobs as the sequence
    return job_order


# Genetic Algorithm

In [23]:
import random

def genetic_algorithm(PROC, population_size=100, generations=500, mutation_rate=0.1):
    N_JOBS = len(PROC)
    
    def create_individual():
        return random.sample(range(N_JOBS), N_JOBS)
    
    def crossover(parent1, parent2):
        crossover_point = random.randint(1, N_JOBS-2)
        child1 = parent1[:crossover_point] + [job for job in parent2 if job not in parent1[:crossover_point]]
        child2 = parent2[:crossover_point] + [job for job in parent1 if job not in parent2[:crossover_point]]
        return child1, child2
    
    def mutate(individual):
        i, j = random.sample(range(N_JOBS), 2)
        individual[i], individual[j] = individual[j], individual[i]
    
    def fitness(individual):
        return totalTime(PROC, individual)
    
    # Initial population
    population = [create_individual() for _ in range(population_size)]
    
    for _ in range(generations):
        # Evaluate fitness of the population
        population = sorted(population, key=fitness)
        
        # Create the next generation
        next_generation = population[:population_size//2]  # Keep top 50%
        
        # Crossover
        for i in range(0, len(next_generation), 2):
            if i+1 < len(next_generation):
                child1, child2 = crossover(next_generation[i], next_generation[i+1])
                next_generation.append(child1)
                next_generation.append(child2)
        
        # Mutate
        for individual in next_generation[1:]:  # Skip the best individual
            if random.random() < mutation_rate:
                mutate(individual)
        
        # Replace old population with the new generation
        population = next_generation[:population_size]
    
    # Return the best individual from the final population
    best_individual = min(population, key=fitness)
    return best_individual


In [24]:
# Implementing Greedy Algorithm
greedy_schedule = greedy_algorithm(PROC)
print("Greedy Algorithm Schedule:", greedy_schedule)
greedy_time = totalTime(PROC, greedy_schedule)
print("Total Time (Greedy):", greedy_time)

# Implementing Genetic Algorithm
genetic_schedule = genetic_algorithm(PROC)
print("Genetic Algorithm Schedule:", genetic_schedule)
genetic_time = totalTime(PROC, genetic_schedule)
print("Total Time (Genetic):", genetic_time)


Greedy Algorithm Schedule: [6, 5, 1, 2, 3, 0, 4]
Total Time (Greedy): 59
Genetic Algorithm Schedule: [5, 4, 6, 3, 1, 0, 2]
Total Time (Genetic): 54


In [25]:
def print_schedule(PROC, job_order):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])

    start_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    for job_index in range(N_JOBS):
        job = job_order[job_index]
        for machine in range(N_MACHINES):
            if machine == 0:
                start_time[job][machine] = stop_time[job_order[job_index-1]][machine] if job_index > 0 else 0
            else:
                start_time[job][machine] = max(stop_time[job][machine-1], stop_time[job_order[job_index-1]][machine] if job_index > 0 else 0)
            stop_time[job][machine] = start_time[job][machine] + PROC[job][machine]

    print("Detailed Schedule:")
    for job in job_order:
        print(f"Job {job}: ", end="")
        for machine in range(N_MACHINES):
            print(f"[Start: {start_time[job][machine]}, Stop: {stop_time[job][machine]}] ", end="")
        print()

# Implementing Greedy Algorithm
greedy_schedule = greedy_algorithm(PROC)
print("Greedy Algorithm Schedule:", greedy_schedule)
greedy_time = totalTime(PROC, greedy_schedule)
print("Total Time (Greedy):", greedy_time)
print_schedule(PROC, greedy_schedule)


Greedy Algorithm Schedule: [6, 5, 1, 2, 3, 0, 4]
Total Time (Greedy): 59
Detailed Schedule:
Job 6: [Start: 0, Stop: 5] [Start: 5, Stop: 7] [Start: 7, Stop: 14] [Start: 14, Stop: 16] 
Job 5: [Start: 5, Stop: 6] [Start: 7, Stop: 14] [Start: 14, Stop: 18] [Start: 18, Stop: 23] 
Job 1: [Start: 6, Stop: 15] [Start: 15, Stop: 18] [Start: 18, Stop: 21] [Start: 23, Stop: 28] 
Job 2: [Start: 15, Stop: 19] [Start: 19, Stop: 21] [Start: 21, Stop: 29] [Start: 29, Stop: 36] 
Job 3: [Start: 19, Stop: 20] [Start: 21, Stop: 28] [Start: 29, Stop: 34] [Start: 36, Stop: 44] 
Job 0: [Start: 20, Stop: 27] [Start: 28, Stop: 34] [Start: 34, Stop: 37] [Start: 44, Stop: 50] 
Job 4: [Start: 27, Stop: 34] [Start: 34, Stop: 40] [Start: 40, Stop: 41] [Start: 50, Stop: 59] 


In [26]:
# Implementing Genetic Algorithm
genetic_schedule = genetic_algorithm(PROC)
print("Genetic Algorithm Schedule:", genetic_schedule)
genetic_time = totalTime(PROC, genetic_schedule)
print("Total Time (Genetic):", genetic_time)
print_schedule(PROC, genetic_schedule)


Genetic Algorithm Schedule: [5, 4, 3, 1, 2, 6, 0]
Total Time (Genetic): 54
Detailed Schedule:
Job 5: [Start: 0, Stop: 1] [Start: 1, Stop: 8] [Start: 8, Stop: 12] [Start: 12, Stop: 17] 
Job 4: [Start: 1, Stop: 8] [Start: 8, Stop: 14] [Start: 14, Stop: 15] [Start: 17, Stop: 26] 
Job 3: [Start: 8, Stop: 9] [Start: 14, Stop: 21] [Start: 21, Stop: 26] [Start: 26, Stop: 34] 
Job 1: [Start: 9, Stop: 18] [Start: 21, Stop: 24] [Start: 26, Stop: 29] [Start: 34, Stop: 39] 
Job 2: [Start: 18, Stop: 22] [Start: 24, Stop: 26] [Start: 29, Stop: 37] [Start: 39, Stop: 46] 
Job 6: [Start: 22, Stop: 27] [Start: 27, Stop: 29] [Start: 37, Stop: 44] [Start: 46, Stop: 48] 
Job 0: [Start: 27, Stop: 34] [Start: 34, Stop: 40] [Start: 44, Stop: 47] [Start: 48, Stop: 54] 


In [27]:
import math

def stochastic_optimization(PROC, initial_temp=1000, cooling_rate=0.99, max_iterations=1000):
    def random_neighbour(sequence):
        new_seq = sequence[:]
        i, j = random.sample(range(len(new_seq)), 2)
        new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
        return new_seq
    
    def acceptance_probability(old_cost, new_cost, temperature):
        if new_cost < old_cost:
            return 1.0
        return math.exp((old_cost - new_cost) / temperature)
    
    # Initial job sequence
    current_sequence = random.sample(range(len(PROC)), len(PROC))
    current_cost = totalTime(PROC, current_sequence)
    best_sequence = current_sequence[:]
    best_cost = current_cost
    
    temperature = initial_temp
    
    for _ in range(max_iterations):
        new_sequence = random_neighbour(current_sequence)
        new_cost = totalTime(PROC, new_sequence)
        
        if acceptance_probability(current_cost, new_cost, temperature) > random.random():
            current_sequence = new_sequence
            current_cost = new_cost
            
        if current_cost < best_cost:
            best_sequence = current_sequence
            best_cost = current_cost
            
        temperature *= cooling_rate
    
    return best_sequence

# Implementing Stochastic Optimization
stochastic_schedule = stochastic_optimization(PROC)
print("Stochastic Optimization Schedule:", stochastic_schedule)
stochastic_time = totalTime(PROC, stochastic_schedule)
print("Total Time (Stochastic):", stochastic_time)
print_schedule(PROC, stochastic_schedule)


Stochastic Optimization Schedule: [5, 4, 2, 1, 3, 6, 0]
Total Time (Stochastic): 54
Detailed Schedule:
Job 5: [Start: 0, Stop: 1] [Start: 1, Stop: 8] [Start: 8, Stop: 12] [Start: 12, Stop: 17] 
Job 4: [Start: 1, Stop: 8] [Start: 8, Stop: 14] [Start: 14, Stop: 15] [Start: 17, Stop: 26] 
Job 2: [Start: 8, Stop: 12] [Start: 14, Stop: 16] [Start: 16, Stop: 24] [Start: 26, Stop: 33] 
Job 1: [Start: 12, Stop: 21] [Start: 21, Stop: 24] [Start: 24, Stop: 27] [Start: 33, Stop: 38] 
Job 3: [Start: 21, Stop: 22] [Start: 24, Stop: 31] [Start: 31, Stop: 36] [Start: 38, Stop: 46] 
Job 6: [Start: 22, Stop: 27] [Start: 31, Stop: 33] [Start: 36, Stop: 43] [Start: 46, Stop: 48] 
Job 0: [Start: 27, Stop: 34] [Start: 34, Stop: 40] [Start: 43, Stop: 46] [Start: 48, Stop: 54] 


In [28]:
import numpy as np

def ant_colony_optimization(PROC, n_ants=50, n_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5):
    N_JOBS = len(PROC)
    
    pheromone_matrix = np.ones((N_JOBS, N_JOBS))
    visibility_matrix = 1.0 / np.array([[sum(PROC[job]) for job in range(N_JOBS)] for _ in range(N_JOBS)])
    
    def choose_next_job(pheromone_row, visibility_row):
        probabilities = (pheromone_row ** alpha) * (visibility_row ** beta)
        probabilities /= probabilities.sum()
        return np.random.choice(range(N_JOBS), p=probabilities)
    
    best_sequence = None
    best_cost = float('inf')
    
    for iteration in range(n_iterations):
        all_sequences = []
        all_costs = []
        
        for _ in range(n_ants):
            sequence = []
            unvisited_jobs = set(range(N_JOBS))
            current_job = random.choice(list(unvisited_jobs))
            sequence.append(current_job)
            unvisited_jobs.remove(current_job)
            
            while unvisited_jobs:
                next_job = choose_next_job(pheromone_matrix[current_job], visibility_matrix[current_job])
                while next_job not in unvisited_jobs:
                    next_job = choose_next_job(pheromone_matrix[current_job], visibility_matrix[current_job])
                sequence.append(next_job)
                unvisited_jobs.remove(next_job)
                current_job = next_job
            
            cost = totalTime(PROC, sequence)
            all_sequences.append(sequence)
            all_costs.append(cost)
            
            if cost < best_cost:
                best_sequence = sequence
                best_cost = cost
        
        # Update pheromones
        pheromone_matrix *= (1 - evaporation_rate)
        for sequence, cost in zip(all_sequences, all_costs):
            for i in range(N_JOBS - 1):
                pheromone_matrix[sequence[i]][sequence[i+1]] += 1.0 / cost
    
    return best_sequence

# Implementing Ant Colony Optimization
aco_schedule = ant_colony_optimization(PROC)
print("Ant Colony Optimization Schedule:", aco_schedule)
aco_time = totalTime(PROC, aco_schedule)
print("Total Time (ACO):", aco_time)
print_schedule(PROC, aco_schedule)


Ant Colony Optimization Schedule: [5, 4, 6, 3, 0, 2, 1]
Total Time (ACO): 54
Detailed Schedule:
Job 5: [Start: 0, Stop: 1] [Start: 1, Stop: 8] [Start: 8, Stop: 12] [Start: 12, Stop: 17] 
Job 4: [Start: 1, Stop: 8] [Start: 8, Stop: 14] [Start: 14, Stop: 15] [Start: 17, Stop: 26] 
Job 6: [Start: 8, Stop: 13] [Start: 14, Stop: 16] [Start: 16, Stop: 23] [Start: 26, Stop: 28] 
Job 3: [Start: 13, Stop: 14] [Start: 16, Stop: 23] [Start: 23, Stop: 28] [Start: 28, Stop: 36] 
Job 0: [Start: 14, Stop: 21] [Start: 23, Stop: 29] [Start: 29, Stop: 32] [Start: 36, Stop: 42] 
Job 2: [Start: 21, Stop: 25] [Start: 29, Stop: 31] [Start: 32, Stop: 40] [Start: 42, Stop: 49] 
Job 1: [Start: 25, Stop: 34] [Start: 34, Stop: 37] [Start: 40, Stop: 43] [Start: 49, Stop: 54] 
